In [1]:
sc.master

'local[*]'

In [29]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import IntegerType

In [2]:
a=[1,2,3]
b=[ab+1 for ab in a]

In [13]:
userData = [["Leo", 16], ["Marry", 21], 
                     ["Jack", 14], ["Tom", 18]]
print(userData)
# 创建测试df
#pyspark  toDF need list[]
userDF = sc.parallelize(userData).toDF(["name", "age"])
#("name", "age")
# 注册一张user表
# userDF.registerTempTable("user")
userDF.createOrReplaceTempView("user")

[['Leo', 16], ['Marry', 21], ['Jack', 14], ['Tom', 18]]


In [17]:
print(userDF.printSchema)
userDF.show()

<bound method DataFrame.printSchema of DataFrame[name: string, age: bigint]>
+-----+---+
| name|age|
+-----+---+
|  Leo| 16|
|Marry| 21|
| Jack| 14|
|  Tom| 18|
+-----+---+



In [20]:
userDF.columns

['name', 'age']

In [ ]:
//注册自定义函数（通过匿名函数）
spark.udf.register("strLen", (str: String) => str.length())
//注册自定义函数（通过实名函数）
spark.udf.register("isAdult", isAdult _)
spark.sql("select *,strLen(name) as name_len,isAdult(age) as isAdult from user").show


In [31]:
# 这里的udf来自于spark sql.function.udf
# 注册自定义函数（通过匿名函数）
strLen = udf(lambda x:len(x))
strLen_2 = udf(lambda x:len(x), IntegerType())
# 注册自定义函数（通过实名函数）
def isAdult(x):
    if x >= 18 :
        return True
    else:
        return False
    
udf_isAdult = udf(isAdult)
userDF.withColumn("name_len", strLen(col("name").cast("String"))) \
        .withColumn("name_len_2", strLen_2(col("name"))).show()
userDF.withColumn("isAdult", udf_isAdult(col("age").cast("Int"))).show()


+-----+---+--------+----------+
| name|age|name_len|name_len_2|
+-----+---+--------+----------+
|  Leo| 16|       3|         3|
|Marry| 21|       5|         5|
| Jack| 14|       4|         4|
|  Tom| 18|       3|         3|
+-----+---+--------+----------+

+-----+---+-------+
| name|age|isAdult|
+-----+---+-------+
|  Leo| 16|  false|
|Marry| 21|   true|
| Jack| 14|  false|
|  Tom| 18|   true|
+-----+---+-------+

